# <center>**Work 4: Transformação Geometricas**</center>
# <center>Wladimir Arturo Garces Carrillo</center>
### <center>RA204059</center>
#### <center>MO443_1s2022</center>

# **Considerações iniciais**


## **Entradas**


>#### São considerados para entrada:
>
>>- Ânguo de rotação medido em graus no  sentido anti-horario.
>>- Factor de escala em x.
>>- Factor de escala em y.
>>- Método de interpolação utilizado.
>>- Imagem de entrada no formato PNG.
>>- Imagem de saída no formato PNG (após a transformação geométrica).
>>- Modo de operação, tipo de transformação.


## **Formato de entrada**


<center>python run.py -a |ãngulo| -sx |X scale| -sy |Y scale| -m |interpolação| -i |img_i| -o |img_o| -mod |Mode|*</center>


>
>#### **Angulo (-a, --angulo):**
>Refere-se ao ãnguo de rotação medido em graus no sentido anti-horario que vai se aplicar na transformação geométrica da imgem.
>
>#### **X Scale (-sx, --x_scale):**
>Refere-se ao factor de escala noi eixo "X" que vai se aplicar na transformação geométrica da imgem.
>
>#### **Y Scale (-sy, --y_scale):**
>Refere-se ao factor de escala noi eixo "Y" que vai se aplicar na transformação geométrica da imgem.
>
>#### **Interpolação (-m, --interp):**
>Refere-se ao modo em que vai se calcular as intensidade dos pixels após a transformação geométrica.
>>- Interpolação pelo Vizinho Mais Proximo (inter_VMP).
>>- Interpolação Bilinear (inter_bilinear)
>
>#### **Img_i (-i, --img_i):**
>Refere-se ao nome do arquivo de imagem que será fornecido ao sistema como uma string. Deve-se considerar que o nome do arquivo tem que ficar contido na pasta *img/* do projeto. O nome contém a extensão do arquivo em *.png* ou *.jpg*.
>
>#### **Img_o (-o, --img_o):**
>Refere-se ao nome do arquivo de imagem que será devolvida e guardada por o sistema em formato *.png*. Deve-se considerar que o arquivo com este nome reside na pasta *results/* e que o nome contém a extensão do arquivo em *.png*.
>
>#### **Mode (-mod, --mode):**
>Refere-se ao modo da transformação geométrica que vai se aplicar na imagem.
>>- Perspectiva.
>>- Escala.
>>- Rotação.
>
> ___

# **Uso**


>Os pacotes necessários para executar corretamente este arquivo são numpy, matplotlib, scikit-learn e opencv, que são instalados com os seguintes comandos de um console.

<center>pip install numpy</center> 
<center>pip install opencv-python</center>

# **Processo**

>O processo do programa é descrito abaixo
>>- Carrega da imgem.
>>- Calcular transformação.
>>- Guardar resultados.
>
>O processo descrito acima é encontrado na função de python a seguir.

In [ ]:
def main(args):
    img = imagem_upload(args)
    img_s = modo[args["mode"]](img, args)
    show_save_img(img, img_s, args)

# Desenvolvimento

## **Perspectiva**



In [ ]:
def perspectiva(img, args):
    # This function applies a perspective transformation to an image.
    # Init points
    pts1 = np.float32([[37,51],[342,42],[485,467],[73,380]])
    # End points
    pts2 = np.float32([[0,0],[511,0],[511,511],[0,511]])
    
    # Calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(pts1,pts2)
    # Apply the perspective transform to the image
    img_s = cv2.warpPerspective(img, M, (511,511))
    
    img[51,27:47] = [255,0,0]
    img[41:61,37] = [255,0,0]
    
    img[42,332:352] = [0,255,0]
    img[32:52,342] = [0,255,0]
    
    img[467,475:495] = [0,0,255]
    img[457:477,485] = [0,0,255]
    
    img[380,63:83] = [255,255,255]
    img[370:390,73] = [255,255,255]
    
    return img_s

## Escala

In [ ]:
def escala(img, args):
    # This function scales an image around the center of the image.
    # Create out image whit the same size of the original image.
    # Calculing midle of the image.
    # Calculing a scale matrix.
    # Calculing inverse scale matrix.
    # Scale the image process.
    
    sx = args["x_scale"]
    sy = args["y_scale"]
    
    img_s = np.zeros(img.shape, dtype=np.uint8)
    y, x = img_s.shape[:2]
    midy, midx = y//2, x//2
    
    s = np.array([[sx, 0, 0],
                  [0, sy, 0],
                  [0, 0, 1]])
    
    i_s = np.linalg.inv(s)
    for i in range(y-1):
        for j in range(x-1):
            xnew = j - midx
            ynew = i - midy
            p = np.array([[xnew],
                          [ynew],
                          [1]])
            # Scale the point
            p = np.dot(i_s, p)
            # Add the midle of the image
            p[0] += midx
            p[1] += midy
            # If the point is inside the image
            if p[0]>=0 and p[0]<x-1 and p[1]>=0 and p[1]<y-1:
                # img_s[i,j] = inter_bilinear(img, p) or inter_VMP(img, p)
                img_s[i,j] = interpolador(args)(img, p)
    return img_s

## Rotação

In [ ]:
def rotate(img, args):
    # This function rotates an image around the center of the image.
    # Create out image whit the same size of the original image.
    # Calculing midle of the image.
    # Calculing a rotation matrix.
    # Calculing inverse rotation matrix.
    # Rotate the image process.
    angulo = args["angulo"]
    salida = np.zeros(img.shape, dtype=np.uint8)
    y, x = salida.shape[:2]
    midy, midx = y//2, x//2
    cos = np.cos(np.deg2rad(angulo))
    sen = np.sin(np.deg2rad(angulo))
    r = np.array([[cos, -sen, 0], 
                [sen,  cos, 0],
                [  0,    0, 1]])
    ir = np.linalg.inv(r)

    for i in range(y-1):
        for j in range(x-1):
            xnew = j - midx
            ynew = i - midy
            p = np.array([[xnew],
                          [ynew],
                          [1]])
            # Rotate the point
            p = np.dot(ir, p)
            # Add the midle of the image
            p[0] += midx
            p[1] += midy
            # If the point is inside the image
            if p[0]>=0 and p[0]<x-1 and p[1]>=0 and p[1]<y-1:
                # salida[i,j] = inter_bilinear(img, p) or inter_VMP(img, p)
                salida[i,j] = interpolador(args)(img, p)
    return salida

## Interpolador

### Interpolação pelo Vizinho Mais Próximo

In [ ]:
def inter_VMP(img, p):
    # print(p[0])
    p = p.astype(int)
    return img[p[1],p[0]]

Esta função arredonda os valores para o vizinho mais próximo.

### Interpolação Biliniar

In [ ]:
def inter_bilinear(img, p):
    dx = p[0] - p[0].astype(int)
    dy = p[1] - p[1].astype(int)
    p = p.astype(int)
    t1 = (1-dx)*(1-dy)*img[p[1],p[0]]
    t2 = dx*(1-dy)*img[p[1],p[0]+1]
    t3 = dx*dy*img[p[1]+1,p[0]+1]
    t4 = (1-dx)*dy*img[p[1]+1,p[0]]
    intensidade = t1+t2+t3+t4
    return intensidade

Nesta função são calculadas as diferenças entre os pixels da imagem e aqueles calculados após a transformação, terminando na soma da multiplicação desses fatores pelas intensidades dos vizinhos do pixel resultante, ao final o resultado é assumido como a intensidade do novo pixel.

# Experimentos

### Rotação

##### Interpolação bilinear

In [7]:
!python run.py -mod rotate -a 45 -i baboon.png -m inter_bilinear

{'angulo': 45.0, 'x_scale': 1, 'y_scale': 1, 'altura': 0, 'interp': 'inter_bilinear', 'img_i': 'baboon.png', 'img_o': None, 'mode': 'rotate'}


<center>
    <figure>
        <div>
            <img src="results/rotate_45.0.png"
                 style="width:50%;height:50%"
                 title="rotate_45.0"/>
        <figcaption>Fig 1. Resultados da rotação em 45º com Interpolador Bilinear</figcaption>
        </div>
    </figure>
</center>

##### Interpolação pelo Vizinho Mais Próximo

In [9]:
!python run.py -mod rotate -a 80 -i baboon.png -m inter_VMP

{'angulo': 80.0, 'x_scale': 1, 'y_scale': 1, 'altura': 0, 'interp': 'inter_VMP', 'img_i': 'baboon.png', 'img_o': None, 'mode': 'rotate'}


<center>
    <figure>
        <div>
            <img src="results/rotate_80.0.png"
                 style="width:50%;height:50%"
                 title="rotate_80.0"/>
        <figcaption>Fig 2. Resultados da rotação em 80º com Interpolador VMP.</figcaption>
        </div>
    </figure>
</center>

A interpolação VMP fornece u m bom resultado, com intensidades dos pixels ótimas, porem a interpolação Bilinear conserva melhor a suavidade das bordas como pode se observar nas Fig 1 e Fig 2. Nesta ultima as bordas da nariz (em Vermelho) ficam com resolução baixa, mas na Fig 1, esto não se olha.

### Escala

##### Interpolação Bilinear

In [10]:
!python run.py -mod escala -sx 2 -sy 2 -i baboon.png -m inter_bilinear

{'angulo': 0, 'x_scale': 2.0, 'y_scale': 2.0, 'altura': 0, 'interp': 'inter_bilinear', 'img_i': 'baboon.png', 'img_o': None, 'mode': 'escala'}


<center>
    <figure>
        <div>
            <img src="results/escala_2.0_2.0.png"
                 style="width:50%;height:50%"
                 title="escala_2.0_2.0"/>
        <figcaption>Fig 3. Resultados da escala em sx=2 e sy=2 com Interpolador Bilinear</figcaption>
        </div>
    </figure>
</center>

##### Interpolação pelo Vizinho Mais Próximo

In [11]:
!python run.py -mod escala -sx 2 -sy 2 -i baboon.png -o escala_VMP_2.0_2.0.png -m inter_VMP

{'angulo': 0, 'x_scale': 2.0, 'y_scale': 2.0, 'altura': 0, 'interp': 'inter_VMP', 'img_i': 'baboon.png', 'img_o': 'escala_VMP_2.0_2.0.png', 'mode': 'escala'}


<center>
    <figure>
        <div>
            <img src="results/escala_VMP_2.0_2.0.png"
                 style="width:50%;height:50%"
                 title="escala_VMP_2.0_2.0"/>
        <figcaption>Fig 4. Resultados da escala em sx=2 e sy=2 com Interpolador VMP</figcaption>
        </div>
    </figure>
</center>

Novamente olhamos nas Fig 3 e Fig 4 que tem muitas diferencias nas intensedades dos pixels. Fica muito melhor os resultados fornecidos por a interpolação Bilinear.

##### Outras escalas

In [12]:
!python run.py -mod escala -sx 0.5 -sy 0.5 -i baboon.png -m inter_bilinear

{'angulo': 0, 'x_scale': 0.5, 'y_scale': 0.5, 'altura': 0, 'interp': 'inter_bilinear', 'img_i': 'baboon.png', 'img_o': None, 'mode': 'escala'}


<center>
    <figure>
        <div>
            <img src="results/escala_0.5_0.5.png"
                 style="width:50%;height:50%"
                 title="escala_0.5_0.5"/>
        <figcaption>Fig 5. Resultados da escala em sx=0.5 e sy=0.5 com Interpolador Bilinear</figcaption>
        </div>
    </figure>
</center>

In [13]:
!python run.py -mod escala -sx 0.5 -sy 1 -i baboon.png -m inter_bilinear

{'angulo': 0, 'x_scale': 0.5, 'y_scale': 1.0, 'altura': 0, 'interp': 'inter_bilinear', 'img_i': 'baboon.png', 'img_o': None, 'mode': 'escala'}


<center>
    <figure>
        <div>
            <img src="results/escala_0.5_1.0.png"
                 style="width:50%;height:50%"
                 title="escala_0.5_1.0"/>
        <figcaption>Fig 6. Resultados da escala em sx=0.5 e sy=1 com Interpolador Bilinear</figcaption>
        </div>
    </figure>
</center>

# Conclusão

>- Resultados ótimos são alcançados com ambas as abordagens.
>- Quanto mais complexo for o cálculo das intensidades dos pixels, melhor será a qualidade da imagem resultante.